# FoodData Central (FDC) API in Python

by Sebastian Shirk

**FDC API Guide**: https://fdc.nal.usda.gov/api-guide.html#bkmk-5

**FDC API Documentation**: https://app.swaggerhub.com/apis/fdcnal/food-data_central_api/1.0.1

The FoodData Central (FDC) API provides programmatic access to the FDC database, which is maintained by the U.S. Department of Agriculture (USDA). The database contains a wide range of information about food products, including nutrient content, ingredients, and serving sizes.

"U.S. Department of Agriculture, Agricultural Research Service. FoodData Central, 2019. fdc.nal.usda.gov."

"USDA FoodData Central data are in the public domain and they are not copyrighted. They are published under [CC0 1.0 Universal (CC0 1.0)](https://creativecommons.org/publicdomain/zero/1.0/).

*These reciple examples were tested on August 23, 2024.*

## Setup

### Import Libraries

This tutorial will use the following libraries:

In [3]:
import requests
import polars as pl
from pprint import pprint

### API Key

For this API you need an API key. You can get one by signing up for free at https://fdc.nal.usda.gov/api-key-signup.html.

Save your API key in a file called `api_key.py` and import your key.

In [4]:
from api_key import my_key

### Setting variables, parameters, and calling the API

First, we need to set up the two parameters that we will use to call the API:
* The food name you are searching for
* The number of results you want to display.

In [5]:
search_url = f'https://api.nal.usda.gov/fdc/v1/foods/search'
query = ""
pageSize = 10

# Change the query to search for different foods and the pageSize to get more or less results
query = 'cheese'
pageSize = '2'

search_params = {
    "query": query,
    "dataType": "Branded",
    "pageSize": pageSize,
    "pageNumber": 0,
    "sortBy": "dataType.keyword",
    "sortOrder": "asc",
    "brandOwner": "",
    "api_key": my_key,
}

# Used to get all the foods of that category in the database
response = requests.get(search_url, params=search_params)
results = response.json().get('foods', [])

# Used to look up each individual food to get more details
def get_food_details(FDCID, api_key):
    url = f'https://api.nal.usda.gov/fdc/v1/food/{FDCID}'
    params = {'api_key': api_key}
    response = requests.get(url, params=params)
    return response.json()

## 1. Creating a Nutrient Table

The function below creates a table of nutrients for a given food item. 

In [14]:
def get_nutrient_table():
    food_data = []
    nutrient_data = []

    for food in results:
        FDCID = food.get('fdcId')
        details = get_food_details(FDCID, my_key)
        brand_name = details.get('brandName', '')
        brand_owner = details.get('brandOwner', '')
        marketCountry = details.get('marketCountry', '')
        description = details.get('description', '')
        food_category = details.get('brandedFoodCategory', '')
        serving_size = details.get('servingSize', '')
        serving_size_unit = details.get('servingSizeUnit', '')

        food_data.append([
            FDCID, brand_name, brand_owner, marketCountry, description, food_category, ""
        ])

        nutrient_data.append([
            FDCID, "Serving Size", f"{serving_size} {serving_size_unit}"
        ])

        for nutrient in details.get('foodNutrients', []):
            nutrient_name = nutrient.get('nutrient', {}).get('name', '')
            nutrient_amount = nutrient.get('amount', '')
            nutrient_unit = nutrient.get('nutrient', {}).get('unitName', '')
            
            nutrient_data.append([
                FDCID, nutrient_name, f"{nutrient_amount} {nutrient_unit}"
            ])

    food_df = pl.DataFrame(
        food_data,
        schema=["FDCID", "Brand Name", "Brand Owner", "Market Country", "Description", "Food Category", "Calculated from value per serving size measure"],
        orient="row"
    )

    nutrient_df = pl.DataFrame(
        nutrient_data,
        schema=["FDCID", "Nutrient Name", "Nutrient Amount"],
        orient="row"
    )

    nutrient_pivot_df = nutrient_df.pivot(
        index="FDCID",
        on="Nutrient Name",
        values="Nutrient Amount"
    )

    combined_df = food_df.join(nutrient_pivot_df, on="FDCID", how="left")
    pl.Config.set_fmt_str_lengths(1000)
    pl.Config.set_tbl_cols(8)
    # Uncomment the following line to see all columns
    # pl.Config.set_tbl_cols(100)
    pl.Config.set_tbl_rows(100)  

    print(combined_df)

get_nutrient_table()

shape: (2, 22)
┌─────────┬────────────┬────────────┬────────────┬───┬───────────┬───────────┬─────────┬───────────┐
│ FDCID   ┆ Brand Name ┆ Brand      ┆ Market     ┆ … ┆ Fatty     ┆ Calcium,  ┆ Protein ┆ Cholester │
│ ---     ┆ ---        ┆ Owner      ┆ Country    ┆   ┆ acids,    ┆ Ca        ┆ ---     ┆ ol        │
│ i64     ┆ str        ┆ ---        ┆ ---        ┆   ┆ total     ┆ ---       ┆ str     ┆ ---       │
│         ┆            ┆ str        ┆ str        ┆   ┆ saturated ┆ str       ┆         ┆ str       │
│         ┆            ┆            ┆            ┆   ┆ ---       ┆           ┆         ┆           │
│         ┆            ┆            ┆            ┆   ┆ str       ┆           ┆         ┆           │
╞═════════╪════════════╪════════════╪════════════╪═══╪═══════════╪═══════════╪═════════╪═══════════╡
│ 1943515 ┆ FERNDALE   ┆ Ferndale   ┆ United     ┆ … ┆ 17.61 g   ┆ 0.0 mg    ┆ 21.13 g ┆ 53.0 mg   │
│         ┆ FARMSTEAD  ┆ Farmstead  ┆ States     ┆   ┆           ┆          

## 2. Creating an Ingredient Table

The function below will create a table of ingredients for a given food product.

In [16]:
def get_ingredients_table():
    food_data = []

    for food in results:
        FDCID = food.get('fdcId')
        details = get_food_details(FDCID, my_key)
        marketCountry = details.get('marketCountry', '')
        brand_name = details.get('brandName', '')
        brand_owner = details.get('brandOwner', '')
        description = details.get('description', '')
        food_category = details.get('brandedFoodCategory', '')
        ingredients = details.get('ingredients', '')

        food_data.append([
            FDCID, brand_name, brand_owner, marketCountry, description, food_category, ingredients
        ])

    food_df = pl.DataFrame(
        food_data,
        schema=["FDCID", "Brand Name", "Brand Owner", "Market Country", "Description", "Food Category", "Ingredients"],
        orient="row"
    )

    pl.Config.set_fmt_str_lengths(10000)
    pl.Config.set_tbl_cols(100)
    pl.Config.set_tbl_rows(100)

    print(food_df)

get_ingredients_table()

shape: (2, 7)
┌─────────┬────────────┬─────────────────┬─────────┬─────────────┬────────────────┬────────────────┐
│ FDCID   ┆ Brand Name ┆ Brand Owner     ┆ Market  ┆ Description ┆ Food Category  ┆ Ingredients    │
│ ---     ┆ ---        ┆ ---             ┆ Country ┆ ---         ┆ ---            ┆ ---            │
│ i64     ┆ str        ┆ str             ┆ ---     ┆ str         ┆ str            ┆ str            │
│         ┆            ┆                 ┆ str     ┆             ┆                ┆                │
╞═════════╪════════════╪═════════════════╪═════════╪═════════════╪════════════════╪════════════════╡
│ 1943515 ┆ FERNDALE   ┆ Ferndale        ┆ United  ┆ CHEESE      ┆ Cheese         ┆ MILK, SALT,    │
│         ┆ FARMSTEAD  ┆ Farmstead LLC   ┆ States  ┆             ┆                ┆ CULTURES,      │
│         ┆            ┆                 ┆         ┆             ┆                ┆ ENZYMES.       │
│ 2083541 ┆ S.J. FALBO ┆ American Pride  ┆ United  ┆ CHEESE      ┆ Cheese    

## 3. Specific Food Item Lookup

These functions will allow us to retrieve information about specific food items from either of the tables we created earlier via the FDCID.

Note that these functions rely on at least one of the tables to be created in the previous steps.

In [17]:
def get_FDCID(idx):
    if 0 <= idx < len(search_params['pageSize']):
        food = results[idx]
        fdc_id = food.get('fdcId')
        print(f"FDCID of idx {idx}: {fdc_id}")
        return fdc_id
    return None

def get_FDCID_data(FDCID):
    if FDCID:
        response = requests.get(f'https://api.nal.usda.gov/fdc/v1/food/{FDCID}', params={'api_key': my_key})
        pprint(response.json())

get_FDCID_data(get_FDCID(0))

FDCID of idx 0: 1943515
{'availableDate': '8/11/2018',
 'brandName': 'FERNDALE FARMSTEAD',
 'brandOwner': 'Ferndale Farmstead LLC',
 'brandedFoodCategory': 'Cheese',
 'dataSource': 'LI',
 'dataType': 'Branded',
 'description': 'CHEESE',
 'discontinuedDate': '',
 'fdcId': 1943515,
 'foodAttributes': [{'id': 2090445,
                     'name': 'Added Package Weight',
                     'value': 9}],
 'foodClass': 'Branded',
 'foodComponents': [],
 'foodNutrients': [{'amount': 176.0,
                    'foodNutrientDerivation': {'code': 'LCCD',
                                               'description': 'Calculated from '
                                                              'a daily value '
                                                              'percentage per '
                                                              'serving size '
                                                              'measure',
                                               'id': 75}